## Simpson Character Classifier

The dataset are accessible through
https://www.kaggle.com/datasets/alexattia/the-simpsons-characters-dataset


Aim: Build a CNN model that could accurately identify character's name inside an image. Challenges arise in this project mainly comes from the randomness of the dataset where for each character, they are presented in a variety of situations with different backgrounds, different angles, etc.

In [1]:
import tensorflow as tf 
import numpy as np            
import matplotlib.pyplot as plt         
import os 
import copy 
import itertools 
from tensorflow import keras 
from keras.utils.np_utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator             
from PIL import Image             
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

Using TensorFlow backend.


## Loading Data

In [1]:
path_train = '/content/drive/MyDrive/Detecting_Character/detecting_character_dataset/simpsons_dataset'
path_test = '/content/drive/MyDrive/Detecting_Character/detecting_character_dataset/kaggle_simpson_testset'

In [ ]:
classes = len(os.listdir(path_train)) # How many classes are there?
characters = sorted(os.listdir(path_train)) # List of characters
print(classes) 